In [12]:
# Run only one feature
%cd /data/bruingjde/on-going/SNAM2021-code/
import src.get_features
import numpy as np

time_str = 'lin'
agg_str = 'q0'
time_func = src.get_features.lin
agg_func = np.min

src.get_features.do_feature(
  feature_func=src.get_features.jc_time_aware, 
  path='data/18/', 
  out_file=f'jc_{time_str}_{agg_str}.npy',
  time_strategy=time_func,
  aggregation_strategy=agg_func
)

/data/bruingjde/on-going/SNAM2021-code


[06/02/21 10:19:46] DEBUG    Get edgelist                                 ]8;id=1622621986.9249506-320004;file:///data/bruingjde/on-going/SNAM2021-code/src/get_features.py\get_features.py]8;;\:194

                    DEBUG    Get instances                                ]8;id=1622621986.9316938-74150;file:///data/bruingjde/on-going/SNAM2021-code/src/get_features.py\get_features.py]8;;\:199

                    DEBUG    Get scores                                   ]8;id=1622621986.9729326-254426;file:///data/bruingjde/on-going/SNAM2021-code/src/get_features.py\get_features.py]8;;\:204

[06/02/21 10:19:47] DEBUG    Save                                         ]8;id=1622621987.9567153-106874;file:///data/bruingjde/on-going/SNAM2021-code/src/get_features.py\get_features.py]8;;\:206

In [10]:
%cd /data/bruingjde/on-going/SNAM2021-code/
import os, json
for directory in os.scandir('data'):
  for subdirectory in os.scandir(directory.path):
    if subdirectory.name != '+000':
      os.rename(subdirectory.path, subdirectory.path + 'a')

/data/bruingjde/on-going/SNAM2021-code


In [14]:
%cd /data/bruingjde/on-going/SNAM2021-code/

import os, shutil
for directory in os.scandir('data'):
  for subdirectory in os.scandir(directory.path):
    for file in os.scandir(subdirectory.path):
      if file.name.endswith('.int'):
        os.remove(file.path)

/data/bruingjde/on-going/SNAM2021-code


In [12]:
import itertools
import numpy as np
iters = list(itertools.product(np.arange(1, 31), np.arange(-100, 101, 20)))
[f'data/{network:02}/{nswap_perc:+04.0f}' for network, nswap_perc in iters]

['data/01/-100',
 'data/01/-080',
 'data/01/-060',
 'data/01/-040',
 'data/01/-020',
 'data/01/+000',
 'data/01/+020',
 'data/01/+040',
 'data/01/+060',
 'data/01/+080',
 'data/01/+100',
 'data/02/-100',
 'data/02/-080',
 'data/02/-060',
 'data/02/-040',
 'data/02/-020',
 'data/02/+000',
 'data/02/+020',
 'data/02/+040',
 'data/02/+060',
 'data/02/+080',
 'data/02/+100',
 'data/03/-100',
 'data/03/-080',
 'data/03/-060',
 'data/03/-040',
 'data/03/-020',
 'data/03/+000',
 'data/03/+020',
 'data/03/+040',
 'data/03/+060',
 'data/03/+080',
 'data/03/+100',
 'data/04/-100',
 'data/04/-080',
 'data/04/-060',
 'data/04/-040',
 'data/04/-020',
 'data/04/+000',
 'data/04/+020',
 'data/04/+040',
 'data/04/+060',
 'data/04/+080',
 'data/04/+100',
 'data/05/-100',
 'data/05/-080',
 'data/05/-060',
 'data/05/-040',
 'data/05/-020',
 'data/05/+000',
 'data/05/+020',
 'data/05/+040',
 'data/05/+060',
 'data/05/+080',
 'data/05/+100',
 'data/06/-100',
 'data/06/-080',
 'data/06/-060',
 'data/06/-040

In [12]:
# Check all features
import os
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import joblib

paths = [p.path for p in os.scandir('data')]  
 
class ProgressParallel(joblib.Parallel):
  def __init__(self, use_tqdm=True, total=None, desc=None, unit='it', *args, 
               **kwargs):
    self._use_tqdm = use_tqdm
    self._total = total
    self._desc = desc
    self._unit = unit
    super().__init__(*args, **kwargs)

  def __call__(self, *args, **kwargs):
    with tqdm(disable=not self._use_tqdm, total=self._total, 
              desc=self._desc, unit=self._unit, leave=False, 
              miniters=1) as self._pbar:
      return joblib.Parallel.__call__(self, *args, **kwargs)

  def print_progress(self):
    if self._total is None: 
      self._pbar.total = self.n_dispatched_tasks
    self._pbar.n = self.n_completed_tasks
    self._pbar.refresh()   
    
def check(path):
  instances = np.array(
    [i for i in pd.read_pickle(os.path.join(path, 'samples.pkl')).index]
  )  
  files = [p.path for p in os.scandir(path + '/features')]
  for file in files:
    scores = np.load(file)
    assert len(instances) == len(scores)
    assert scores.ndim == 1 
    
ProgressParallel(total=30, n_jobs=30)(joblib.delayed(check)(path) 
                                      for path in paths)

  0%|          | 0/30 [00:00<?, ?it/s]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [9]:
# Check instances
import joblib
import pandas as pd
import networkx as nx
import numpy as np
from tqdm.auto import tqdm

def check(index):
  edgelist = pd.read_pickle(f'data/{index:02}/edgelist.pkl').loc[
    lambda x: x['source'] != x['target']] # Do not allow selfloops
  
  edgelist_mature = edgelist.loc[
    lambda x: (x['phase'] == 'mature'), ['source', 'target']]
  graph_mature = nx.from_pandas_edgelist(edgelist_mature)
  
  instances = np.array(
    [i for i in pd.read_pickle(f'data/{index:02}/samples.pkl').index]
  )
  
  for u, v in instances:
    assert nx.shortest_path_length(graph_mature, u, v) == 2
  else:
    print('Ok')
    
class ProgressParallel(joblib.Parallel):
  def __init__(self, use_tqdm=True, total=None, desc=None, unit='it', *args, 
               **kwargs):
    self._use_tqdm = use_tqdm
    self._total = total
    self._desc = desc
    self._unit = unit
    super().__init__(*args, **kwargs)

  def __call__(self, *args, **kwargs):
    with tqdm(disable=not self._use_tqdm, total=self._total, 
              desc=self._desc, unit=self._unit, leave=False, 
              miniters=1) as self._pbar:
      return joblib.Parallel.__call__(self, *args, **kwargs)

  def print_progress(self):
    if self._total is None: 
      self._pbar.total = self.n_dispatched_tasks
    self._pbar.n = self.n_completed_tasks
    self._pbar.refresh()
    
ProgressParallel(n_jobs=30, total=30)(
  joblib.delayed(check)(i) for i in np.arange(1, 31)
)

  0%|          | 0/30 [00:00<?, ?it/s]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]